In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, GET
import pandas as pd

In [2]:
# Set sparql endpoint

sparql_endpoint = "https://reservoir.global/v1/views/Public/CONP/graph/sparql"

In [3]:
# Configure sparql wrapper, default request method GET, default response format JSON

def query_sparql(sparql_endpoint, query, query_method=GET, response_format=JSON):
    sparql_wrapper = SPARQLWrapper(sparql_endpoint)
    sparql_wrapper.addCustomHttpHeader("Content-Type", "application/sparql-query")
    sparql_wrapper.setMethod(query_method)
    sparql_wrapper.setReturnFormat(response_format)
    sparql_wrapper.setQuery(query)
    response_object = sparql_wrapper.query()
    if sparql_wrapper.returnFormat == JSON:
        return response_object._convertJSON()
    return response_object.convert()
    

In [4]:
# Create SPARQL query

query_example = """
SELECT ?s ?p ?o
WHERE {?s ?p ?o}
LIMIT 2
"""

In [ ]:
example_02 = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sdo: <https://schema.org/>
SELECT ?s ?o ?label
WHERE {
       ?s sdo:description ?o;
          rdfs:label ?label.
      }
LIMIT 10
"""

In [5]:
# Query sparql endpoint

results = query_sparql(sparql_endpoint, query_example)

In [6]:
# Convert sparql json results into pandas DataFrame

def results_to_dataframe(json_results):
    columns = json_results["head"]["vars"]
    data = []
    for binding in json_results["results"]["bindings"]:
        row_values = []
        for col in columns:
            value = binding.get(col, {})["value"]
            row_values.append(value)
        data.append(row_values)
            
    return pd.DataFrame(data, columns=columns)

print(results_to_dataframe(results))

                                                   s  \
0  https://bluebrain.github.io/nexus/vocabulary/d...   
1  https://bluebrain.github.io/nexus/vocabulary/d...   

                                                   p  \
0  https://bluebrain.github.io/nexus/vocabulary/c...   
1  https://bluebrain.github.io/nexus/vocabulary/c...   

                                                   o  
0  https://bluebrain.github.io/nexus/schemas/reso...  
1                           2020-06-09T14:00:46.977Z  


In [7]:
# Save DataFrame to csv

def dataframe_to_csv(filename="sparql_results"):
    return results_to_dataframe(results).to_csv(f"{filename}.csv")

dataframe_to_csv()